# Ruokavalion optimointi - mitä ruokia ja kuinka paljon pitää syödä, jotta päivittäiset saantisuositukset täyttyvät?

optimoinnin tulos lista kertoimia (grammoina), joiden mukainen määrä kutakin elintarviketta tulee syödä
Finelin tiedoista jätettävä pois lisäravinteet, laihdutusvalmisteet, äidinmaidonkorvikkeet yms.

rajoitukset:
 kutakin ravintoainetta oltava suositusten mukainen määrä
 kaikki kertoimet 0 - 900 (?)
 kertoimien summa alle 5000
 mahdollisimman pieni määrä nollasta poikkeavia kertoimia
 ruokien yhteenlaskettu energiamäärä 2000 kcal (* 4,184)

In [2]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

In [3]:
# pois laskuista jätettävät elintarviketyypit (FUCLASS):
# lastenruoat, äidinmaidonkorvikkeet, ateriankorvikkeet ja lisäravinteet
omitted_food_types = ('BABYFTOT', 
'BABMEATD',
'BABFISHD',
'BABMILPO',
'BABWATPO',
'BABFRUB',
'BABVEGE',
'BABMIFRU',
'BABOTHER',
'MMILK',
'INFMILK',
'CASMILK',
'PREMILK',
'SOYMILK',
'WHEYMILK',
'AMINMILK',
'SPECTOT',
'SPECSUPP',
'MEALREP',
'SPORTFOO',
'SPECFOOD')

In [4]:
def read_files(path:str) -> tuple:
    """This function reads the following csv files and returns 
    a tuple of pandas data structures: 
    component_value_utf-8.csv
    eufdname_FI_utf-8.csv
    food_utf-8.csv
    saantisuositus_2014.csv

    The function also removes data for various supplements,
    since the target is to look at real foods.

    Args:
        path (str): absolute path to csv files

    Returns:
        tuple: Returns a tuple of pandas data structures with the 
        data from the csv files. (component_value, eufdname, food,
        saantisuositus)
    """

    component_value = pd.read_csv(path + "component_value_utf-8.csv", sep=";")
    eufdname = pd.read_csv(path + "eufdname_FI_utf-8.csv", sep=";")
    food = pd.read_csv(path + "food_utf-8.csv", sep=";")
    saantisuositus = pd.read_csv(path + "saantisuositus_2014.csv", sep=";", header=None, names=["EUFDNAME", "name", "mnuori", "maikuinen", "mkeski", "miäkäs", "mvanha", "npieni","nnuori", "naikuinen", "nkeski", "niäkäs", "nvanha"])

    return component_value, eufdname, food, saantisuositus

In [5]:
component_value, eufdname, food, saantisuositus = read_files('/home/pomo/Asiakirjat/Kurssit/Taitotalo_Python-ohjelmoija/python/portfolio/fineli_20/')
print(component_value.head())
print(eufdname.head())
print(food.head())
print(saantisuositus.head())

   FOODID EUFDNAME  BESTLOC ACQTYPE METHTYPE METHIND
0       1    ENERC  1698.30       S        S  MI0107
1       1      FAT     0.00       F        E  MIR003
2       1   CHOAVL    99.90       S        S  MI0181
3       1   CHOCDF    99.88       S        S  MI0131
4       1     PROT     0.00       L        P  MIR003
  THSCODE                              DESCRIPT LANG
0   ENERC               energia, laskennallinen   FI
1  XENERC  energia, laskennallinen ilman kuitua   FI
2     FAT                                 rasva   FI
3  CHOAVL               hiilihydraatti imeytyvä   FI
4  CHOCDF             hiilihydraatti erotuksena   FI
   FOODID                  FOODNAME FOODTYPE PROCESS  EDPORT   IGCLASS  \
0       1                    SOKERI     FOOD     IND     100  SUGARSYR   
1       2  FRUKTOOSI, HEDELMÄSOKERI     FOOD     IND     100  SUGARSYR   
2       3                  SIIRAPPI     FOOD     IND     100  SUGARSYR   
3       4                    HUNAJA     FOOD     RAW     100  SUGARS

In [6]:
def filter_fuclass(dataframe:pd.DataFrame, fuclass_to_remove:tuple) -> pd.DataFrame:
    """Take a pandas dataframe with food data and remove
    lines where the FUCLASS is one of those specified in 
    fuclass_to_remove. Returns the cleaned dataframe.

    Args:
        dataframe (pandas_df): A pandas dataframe with food names and food use class (FUCLASS)
        fuclass_to_remove (tuple): A tuple of FUCLASS values to use for filtering out unwanted data from the dataframe

    Returns:
        pandas_df: The pandas dataframe minus the lines with specified FUCLASS values
    """
    for food_type in fuclass_to_remove:
        dataframe = dataframe[dataframe.FUCLASS != food_type]

    return dataframe

In [ ]:
# ongelma: kaikille ruuille ei ole kaikkien ravinteiden tietoja, joten pitäisi lisätä nollia 
# puuttuvien tietojen kohdalle. Ravinteiden järjestyksen tulee olla sama kaikissa ruuissa.

def fill_in_missing_zeros(dataframe:pd.Dataframe) -> pd.Dataframe:
    """Takes a pandas dataframe (component_value) where each row represents
    the amount of one nutrient in one food (e.g. calcium in milk). 
    Fills in any missing rows with zeroes in the BESTLOC column
    and appropriate data in the other columns (FOODID and EUFDNAME; the other columns do not matter).

    Args:
        dataframe (pandas_df): A pandas dataframe with information on how much of each nutrient foods contain

    Returns:
        pandas_df: The pandas dataframe modified so that it has the same number of nutrient rows for every food.
    """
    sample = []
    for i in range(1,11):
        food = dataframe[dataframe.FOODID == i]
        sample.append(food['EUFDNAME'].tolist())
    
    longest = []
    for l in sample:
        if len(l) > len(longest):
            longest = l
    
    #TODO - miten datan saa sellaiseen muotoon, että saa lisättyä puuttuvien arvojen kohdalle nollat?
    

In [7]:
def filter_components_by_food(component_value:pd.DataFrame, food:pd.DataFrame) -> pd.DataFrame:
    """Takes two pandas dataframes: component_value and food. 
    Returns a filtered version of component_value 
    that only includes rows that have FOODID values found in food
    (since the food dataframe was previously filtered with filter_fuclass()).

    Args:
        component_value (pandas_df): A dataframe with information on how much of each nutrient foods contain
        food (pandas_df): A dataframe with foods

    Returns:
        pandas_df: A filtered version of component_value
    """
    food_ids = food['FOODID'].tolist()
    new_component_value = component_value[component_value.FOODID.isin(food_ids)]
    
    return new_component_value

In [8]:
def limit_components(component_value:pd.DataFrame, nutrient_list:list) -> pd.DataFrame:
    """Takes a pandas dataframe and a list of nutrients to include. 
    Returns a filtered version of the dataframe that only 
    includes rows that have one of the EUFDNAME values found in the list.

    Args:
        component_value (pandas_df): A dataframe with information on how much of each nutrient foods contain
        nutrient_list (list): A list of nutrient codes

    Returns:
        pandas_df: A filtered version of component_value
    """
    new_component_value = component_value[component_value.EUFDNAME.isin(nutrient_list)]
    
    return new_component_value

In [9]:
def get_single_component_values(component_value:pd.DataFrame, component="ENERC") -> list:
    """Takes a pandas dataframe with the nutrient compositions of foods and the EUFDNAME of a single nutrient. 
    Returns the values of a specified component for the foods included.

    Args:
        component_value (pandas_df): A dataframe with information on how much of each nutrient foods contain
        component (string): the EUFDNAME of a nutrient

    Returns:
        list: A list of the values of the specified component (default: ENERC) for the foods
    """
    specified_only = component_value[component_value.EUFDNAME == component]
    
    return specified_only['BESTLOC'].tolist()

In [34]:
def create_matrix_a_eq(component_value:pd.DataFrame, nutrient_list:list) -> list:
    """Takes a pandas dataframe with the nutrient compositions of foods and
    a list of EUFDNAMEs of nutrients. 
    Returns the A_eq matrix (list of lists) for use in the scipy.optimize.linprog function.

    Args:
        component_value (pandas_df): A dataframe with information on how much of each nutrient foods contain
        nutrient_list (list): A list of EUFDNAMEs

    Returns:
        list of lists: The A_eq matrix where each row is the amounts of one nutrient in the various foods.
    """
    A = []
    for nutrient in nutrient_list:
        specified_only = component_value[component_value.EUFDNAME == nutrient]
        A.append(get_single_component_values(component_value, nutrient))
        
    return A

In [45]:
def create_rda_list(rda:pd.DataFrame, nutrient_list:list, target_group:str="nkeski") -> list:
    """Takes a pandas dataframe with the recommended daily allowances for various nutrients
    and a list of EUFDNAMEs of nutrients. 
    Returns the A_eq matrix (list of lists) for use in the scipy.optimize.linprog function.

    Args:
        rda (pandas_df): A dataframe with rda values for nutrients
        nutrient_list (list): A list of EUFDNAMEs
        target_group (str): The age & sex combination of the target group (default: middle-aged women)

    Returns:
        list: The b constraint vector where each value is the rda of a nutrient.
    """
    specified_only = rda[rda.EUFDNAME.isin(nutrient_list)]
    
    return specified_only[target_group].tolist()

In [72]:
print(len(food.index))
testi = food.loc[food['FOODID']<24990]
print(len(testi.index))
testi.head()


4238
1378


,FOODID,FOODNAME,FOODTYPE,PROCESS,EDPORT,IGCLASS,IGCLASSP,FUCLASS,FUCLASSP
0,1,SOKERI,FOOD,IND,100,SUGARSYR,SUGARTOT,SUGADD,SUGARTOT
1,2,"FRUKTOOSI, HEDELMÄSOKERI",FOOD,IND,100,SUGARSYR,SUGARTOT,SUGADD,SUGARTOT
2,3,SIIRAPPI,FOOD,IND,100,SUGARSYR,SUGARTOT,SUGADD,SUGARTOT
3,4,HUNAJA,FOOD,RAW,100,SUGARSYR,SUGARTOT,SUGADD,SUGARTOT
4,5,"MAKEINEN, KARAMELLI",FOOD,IND,100,SWEET,SUGARTOT,SWEET,SUGARTOT


In [12]:
#filter food
filtered_food = filter_fuclass(food, omitted_food_types)
print(len(filtered_food.index))
testi = filtered_food.loc[food['FOODID']==34990]
testi

4150


,FOODID,FOODNAME,FOODTYPE,PROCESS,EDPORT,IGCLASS,IGCLASSP,FUCLASS,FUCLASSP


In [13]:
#filter component_value by foodid
print(len(component_value))
filtered_comp_val = filter_components_by_food(component_value, filtered_food)
print(len(filtered_comp_val))
filtered_comp_val.tail()

307834
301879


,FOODID,EUFDNAME,BESTLOC,ACQTYPE,METHTYPE,METHIND
307829,35887,CHOLE,323.57,S,R,MI0002
307830,35887,STERT,11.43,S,R,MI0002
307831,35887,TRP,486.20,S,R,MI0002
307832,35887,MYRIC,0.00,S,R,MI0002
307833,35887,QUERCE,0.00,S,R,MI0002


In [58]:
# filter component value by nutrient
print(len(filtered_comp_val))
filtered_comp_val2 = limit_components(filtered_comp_val, ['VITA', 'VITB12'])
print(len(filtered_comp_val2))
filtered_comp_val2.tail()

301879
8271


,FOODID,EUFDNAME,BESTLOC,ACQTYPE,METHTYPE,METHIND
307644,35647,VITB12,1.100,S,R,MI0002
307714,35652,VITA,0.000,S,R,MI0002
307718,35652,VITB12,0.050,S,R,MI0002
307788,35887,VITA,25975.429,S,R,MI0002
307792,35887,VITB12,100.000,S,R,MI0002


In [50]:
c = get_single_component_values(filtered_comp_val)
print(len(c))
print(c[:10])

4144
[1698.3, 1696.6, 1261.4, 1382.1, 1487.5, 2270.15, 1358.84, 1375.56, 1294.8, 2033.21]


In [59]:
A = create_matrix_a_eq(filtered_comp_val2, ['VITA', 'VITB12'])
print(len(A[0]))
print(len(A))
print(type(A[0][0]))
print(type(A))

4127
2
<class 'float'>
<class 'list'>


In [60]:
b = create_rda_list(saantisuositus, nutrient_list=['VITA', 'VITB12'], target_group="nkeski")
print(b)

[700.0, 2.0]


In [66]:
res = linprog(c, A_eq=A, b_eq=b)

TypeError: Invalid input for linprog: A_eq must be a 2D array of numerical values

In [78]:
# testi pienemmällä otoksella
#filter food
filtered_food = filter_fuclass(food, omitted_food_types)
print(f'filtered_food length: {len(filtered_food.index)}')

#filter component_value by foodid
print(f'original component_value length: {len(component_value)}')
filtered_comp_val = filter_components_by_food(component_value, filtered_food)
print(f'filtered_comp_val length: {len(filtered_comp_val.index)}')

# filter component value by nutrient
filtered_comp_val2 = limit_components(filtered_comp_val, ['VITA', 'VITB12'])
print(f'filtered_comp_val2 length: {len(filtered_comp_val2.index)}')

c = get_single_component_values(filtered_comp_val) # huomaa: ei filtered_comp_val2, koska siinä ei ole ENERC mukana
print(f'c length: {len(c)}')
print(c[:10])

A = create_matrix_a_eq(filtered_comp_val2, ['VITA', 'VITB12'])
print(f'A[0] length: {len(A[0])}')
print(f'A[1] length: {len(A[1])}')
print(f'A length: {len(A)}')

b = create_rda_list(saantisuositus, nutrient_list=['VITA', 'VITB12'], target_group="nkeski")
print(f'b: {b}')

res = linprog(c, A_eq=A, b_eq=b)

filtered_food length: 4150
original component_value length: 307834
filtered_comp_val length: 301879
filtered_comp_val2 length: 8271
c length: 4144
[1698.3, 1696.6, 1261.4, 1382.1, 1487.5, 2270.15, 1358.84, 1375.56, 1294.8, 2033.21]
A[0] length: 4127
A[1] length: 4144
A length: 2
b: [700.0, 2.0]


TypeError: Invalid input for linprog: A_eq must be a 2D array of numerical values